In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv('data/train_to_colab.csv')

In [3]:
le = LabelEncoder()
train['rendimiento_global'] = le.fit_transform(train['rendimiento_global'])

In [4]:
df = train.copy()

# =====================================================
# 1. EXTRAER AÑO Y SEMESTRE
# =====================================================
df['periodo_academico'] = df['periodo_academico'].astype(int)
df['anio'] = df['periodo_academico'] // 10
df['semestre'] = df['periodo_academico'] % 10

# =====================================================
# 2. FEATURES POR PROGRAMA ACADÉMICO
# =====================================================
program_group = df.groupby('e_prgm_academico')

df['prog_mean_indicador_1'] = df['e_prgm_academico'].map(program_group['indicador_1'].mean())
df['prog_mean_indicador_2'] = df['e_prgm_academico'].map(program_group['indicador_2'].mean())
df['prog_mean_indicador_3'] = df['e_prgm_academico'].map(program_group['indicador_3'].mean())
df['prog_mean_indicador_4'] = df['e_prgm_academico'].map(program_group['indicador_4'].mean())


# =====================================================
# 3. FEATURES POR DEPARTAMENTO
# =====================================================
dept_group = df.groupby('e_prgm_departamento')

df['dept_mean_indicador'] = df['e_prgm_departamento'].map(
    dept_group[['indicador_1','indicador_2','indicador_3','indicador_4']].mean().mean(axis=1)
)


# =====================================================
# 4. INDICADORES AGREGADOS
# =====================================================
df['indicador_promedio'] = df[['indicador_1','indicador_2','indicador_3','indicador_4']].mean(axis=1)
df['indicador_std']      = df[['indicador_1','indicador_2','indicador_3','indicador_4']].std(axis=1)

df['indicador_234_mean'] = df[['indicador_2','indicador_3','indicador_4']].mean(axis=1)
df['indicador_234_std']  = df[['indicador_2','indicador_3','indicador_4']].std(axis=1)

# =====================================================
# 5. RATIOS ENTRE INDICADORES (CON BASE EN CORRELACIONES)
# =====================================================
df['ratio_1_2'] = df['indicador_1'] / (df['indicador_2'] + 1)
df['ratio_1_3'] = df['indicador_1'] / (df['indicador_3'] + 1)
df['ratio_1_4'] = df['indicador_1'] / (df['indicador_4'] + 1)

df['ratio_1_234'] = df['indicador_1'] / (df['indicador_234_mean'] + 1)

# Diferencias (los indicadores 2–3–4 están muy correlacionados)
df['diff_2_3'] = df['indicador_2'] - df['indicador_3']
df['diff_2_4'] = df['indicador_2'] - df['indicador_4']
df['diff_3_4'] = df['indicador_3'] - df['indicador_4']

# =====================================================
# 6. INTERACCIONES SOCIO–ECONÓMICAS × INDICADORES
# =====================================================
df['promedio_x_estrato']     = df['indicador_promedio'] * df['f_estratovivienda']
df['promedio_x_internet']    = df['indicador_promedio'] * df['f_tieneinternet']
df['promedio_x_computador']  = df['indicador_promedio'] * df['f_tienecomputador']

df['horas_x_indicadores'] = df['e_horassemanatrabaja'] * df['indicador_promedio']
df['matricula_x_indicadores'] = df['e_valormatriculauniversidad'] * df['indicador_promedio']

# =====================================================
# 7. FEATURES AVANZADOS (DE LA PROPUESTA ANTERIOR)
# =====================================================

# --- Ratios adicionales fuertes ---
df['indicador_ratio_1_4'] = df['indicador_1'] / (df['indicador_4'] + 1)
df['indicador_ratio_2_3'] = df['indicador_2'] / (df['indicador_3'] + 1)

# --- Combinaciones lineales de indicadores correlacionados ---
df['indicador_sum_2_4'] = df['indicador_2'] + df['indicador_4']
df['indicador_diff_2_4'] = df['indicador_2'] - df['indicador_4']

df['indicador_sum_3_4'] = df['indicador_3'] + df['indicador_4']
df['indicador_diff_3_4'] = df['indicador_3'] - df['indicador_4']

# --- Interacciones: estrato × indicadores ---
df['estrato_x_ind1'] = df['f_estratovivienda'] * df['indicador_1']
df['estrato_x_ind2'] = df['f_estratovivienda'] * df['indicador_2']
df['estrato_x_ind3'] = df['f_estratovivienda'] * df['indicador_3']
df['estrato_x_ind4'] = df['f_estratovivienda'] * df['indicador_4']

# --- Horas por estrato ---
df['horas_por_estrato'] = df['e_horassemanatrabaja'] / (df['f_estratovivienda'] + 1)

# --- Variables polinómicas ---
df['indicador_1_sq'] = df['indicador_1']**2
df['indicador_2_sq'] = df['indicador_2']**2
df['indicador_3_sq'] = df['indicador_3']**2
df['indicador_4_sq'] = df['indicador_4']**2


In [ ]:
# ===================================================
# IMPORTS
# ===================================================
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# ===================================================
# 0. LIMPIEZA Y FIX DE TIPOS
# ===================================================

numeric_fix_cols = [
    'e_valormatriculauniversidad',
    'e_horassemanatrabaja',
    'f_estratovivienda',
    'f_tieneinternet',
    'f_educacionpadre',
    'f_tienelavadora',
    'f_tieneautomovil',
    'e_pagomatriculapropio',
    'f_tienecomputador',
    'f_educacionmadre'
]

# Convertir a numérico y rellenar -1
df[numeric_fix_cols] = df[numeric_fix_cols].apply(
    lambda col: pd.to_numeric(col, errors='coerce')
).fillna(-1)


# ===================================================
# 1. DEFINIR CAT_FEATURES COMO CATEGORY
# ===================================================
cat_features = [
    'e_prgm_academico',
    'e_prgm_departamento',
    'f_estratovivienda',
    'f_tieneinternet',
    'f_educacionpadre',
    'f_tienelavadora',
    'f_tieneautomovil',
    'f_tienecomputador',
    'f_educacionmadre'
]

for col in cat_features:
    df[col] = df[col].astype("category")


# ===================================================
# 2. DEFINIR TARGET Y FEATURES
# ===================================================
target = 'rendimiento_global'
cols_leakage = ['indicador_1', 'indicador_2', 'indicador_3', 'indicador_4']

features = [col for col in df.columns if col not in [target] + cols_leakage]

X_full = df[features]
y_full = df[target]


# ===================================================
# 3. SUBSAMPLEO ESTRATIFICADO (SOLUCIÓN AL MEMORYERROR)
#    Usa solo el 30% del dataset
# ===================================================
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.70, random_state=42)

for small_idx, _ in sss.split(X_full, y_full):
    X = X_full.iloc[small_idx]
    y = y_full.iloc[small_idx]

print("Dataset reducido:", X.shape)


# ===================================================
# 4. TRAIN / VALID SPLIT (YA NO CAUSA MEMORYERROR)
# ===================================================
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


# ===================================================
# 5. ONE-HOT SPARSE (MUY EFICIENTE EN RAM)
# ===================================================
X_train = pd.get_dummies(
    X_train,
    columns=cat_features,
    sparse=True,
    drop_first=False
)

X_valid = pd.get_dummies(
    X_valid,
    columns=cat_features,
    sparse=True,
    drop_first=False
)

# Alinear columnas sin copiar todo
X_train, X_valid = X_train.align(X_valid, join='left', axis=1, fill_value=0)


# ===================================================
# 6. RANDOM FOREST OPTIMIZADO PARA PC LOCAL
# ===================================================
model = RandomForestClassifier(
    n_estimators=120,
    max_depth=18,
    min_samples_split=3,
    min_samples_leaf=2,
    max_features='sqrt',
    bootstrap=True,           # usa menos RAM que bootstrap=False
    class_weight='balanced',
    n_jobs=-1,                # usa todos los cores disponibles
    random_state=42
)

model.fit(X_train, y_train)


# ===================================================
# 7. MÉTRICAS
# ===================================================
preds = model.predict(X_valid)
acc = accuracy_score(y_valid, preds)

print("\n========================================")
print("ACCURACY VALID:", acc)
print("========================================\n")


# ===================================================
# 8. IMPORTANCIA DE FEATURES
# ===================================================
importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=False)

print(importances.head(25))


MemoryError: Unable to allocate 4.23 MiB for an array with shape (554000, 1) and data type float64